In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from utils.print import print_block
from data.dataset import REFLACXWithClinicalDataset
from utils.gradcam import get_df_label_pred_img_input_loss, show_gradCAMpp_result

from tqdm.notebook import tqdm
from utils.plot import relabel_ellipse_df, get_ellipses_patch
from utils.transform import TransformFuncs
from copy import copy
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import torch
from model.xami import XAMIMultiCocatModal
from utils.gradcam import get_df_label_pred_img_input_loss, show_gradCAMpp_result
from utils.train import get_aus_loss
import shutil


In [2]:
pd.options.mode.chained_assignment = None

# checking if the GPU is available
use_gpu = torch.cuda.is_available()
device = 'cuda' if use_gpu else 'cpu'
print(f"Will be using {device}")

Will be using cuda


In [3]:
reflacx_dataset = REFLACXWithClinicalDataset(image_size=256)

Positive Loss weight:
[0.768546   0.78635013 0.7937685  0.8338279  0.87388724]
Negative Loss weight:
[0.231454   0.21364985 0.20623146 0.1661721  0.12611276]
Random Loss:
0.21026036153991162


In [8]:
len(reflacx_dataset.df['dicom_id'].unique())

590

In [4]:
cxr_prefix = "cxr"
cxr_clinical_prefix ="cxr_clinical"

cxr_model_name = 'test_0_8162_epoch300_WithoutClincal_dim32_2022-02-07 21_43_31_353207'
cxr_clinical_model_name = 'test_0_8260__epoch300_WithClincal_dim32_2022-02-08 10_03_56_953198'

In [5]:
experiment_df = pd.read_csv('Experiment_Image_Only.csv')

In [6]:
# we first get the folder to save the result.
result_saving_destination = './experiment_gramcam_with_diseases'
os.makedirs(result_saving_destination, exist_ok=True) 

for _, case in experiment_df.iterrows():

    # get the positive diseases. 
    positive_diseases = [ col  for col in  reflacx_dataset.labels_cols if case[col] ]

    for model_path in [cxr_model_name, cxr_clinical_model_name]:

        for d in positive_diseases:

            retrieving_folder = f'{model_path}_result'
            destination_folder = os.path.join(result_saving_destination, retrieving_folder, case['ENCODING'])

            os.makedirs(destination_folder, exist_ok=True)

            file_name=  f'{d}_cxr_gradcampp.png'

            grad_cam_image_path = os.path.join(retrieving_folder,case['id'], file_name) 

            shutil.copyfile(grad_cam_image_path, os.path.join    (destination_folder,  file_name))
